# [Sec 0-0] Pytorch trial

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
embedding = nn.Embedding(10, 3)

In [ ]:
embedding.weight

In [ ]:
k=np.random.normal(0, 0.001, (10,3))
embedding.weight = nn.Parameter(torch.from_numpy(k))
embedding.weight

In [ ]:
input = Variable(torch.LongTensor([0,2,4,5]))
input1 = Variable(torch.LongTensor([1]))

In [ ]:
base = embedding(input1)

In [ ]:
a = embedding(input)

In [ ]:
a

## give a try at brocasting

In [ ]:
base * a 

In [ ]:
torch.__version__

In [ ]:
a

# torch matrix multiplication

In [ ]:
torch.t(a)

In [ ]:
print(torch.mm(torch.t(a),a))
print(torch.mm(torch.t(a),a).sum())


In [ ]:
A = torch.mm(torch.t(a),a).float()
W = nn.Parameter(torch.ones(3,3))*2

In [ ]:
print(A)
print(W*A)

In [ ]:
torch.mm(torch.t(a),a).trace()


# [Sec 0-1] Data Pre-processing - hashing

In [1]:
# reading data 
import pandas as pd
X_train = pd.read_csv("data/train.csv")
X_test = pd.read_csv("data/test.csv")

In [2]:
# add time column
X_train['weekday']= (X_train['hour']%10000/100).astype(int)%7
X_train['hour_']= (X_train['hour']%100)
X_test['weekday']= (X_test['hour']%10000/100).astype(int)%7
X_test['hour_']= (X_test['hour']%100)

In [3]:
column2handle = list(X_test.columns.values)[2:]

In [4]:
print(column2handle)

['C1', 'banner_pos', 'site_id', 'site_domain', 'site_category', 'app_id', 'app_domain', 'app_category', 'device_id', 'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'weekday', 'hour_']


In [5]:
hash_list=[]
# will be slowly as list grow up 
def get_hash_value(x):
    try:
        idx = hash_list.index(x)
    except :
        hash_list.append(x)
        idx = len(hash_list)-1
    return idx
    

In [6]:
# dict version
hash_dict=dict()
count =0 
def get_hash_value_dict(x):
    global count
    idx = hash_dict.get(x,count)
    if idx==count:
        hash_dict[x]=count
        count+=1
    return idx

In [7]:
column2handle[0]

'C1'

In [8]:
X_train.columns.values

array(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category',
       'device_id', 'device_ip', 'device_model', 'device_type',
       'device_conn_type', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20',
       'C21', 'weekday', 'hour_'], dtype=object)

### compare 2 implement of hash method

In [9]:
%%time
i='site_id'
X_train[i+'_hash']=X_train[i].apply(get_hash_value)


CPU times: user 2min 34s, sys: 72 ms, total: 2min 34s
Wall time: 2min 34s


In [10]:
%%time
i='site_id'
X_train[i+'_hash']=X_train[i].apply(get_hash_value_dict)

CPU times: user 13.9 s, sys: 48 ms, total: 13.9 s
Wall time: 13.9 s


obviously using dictionary is much suitable here
## build up the hash function

In [11]:
# dict version
hash_dict=dict()
count =0 
def get_hash_value_dict(x):
    global count
    idx = hash_dict.get(x,count)
    if idx==count:
        hash_dict[x]=count
        count+=1
    return idx

In [12]:
%%time
for i in column2handle:
    print (i)
    X_train[i+'_hash']=X_train[i].apply(lambda x :get_hash_value_dict(i+'_'+str(x)))


C1
banner_pos
site_id
site_domain
site_category
app_id
app_domain
app_category
device_id
device_ip
device_model
device_type
device_conn_type
C14
C15
C16
C17
C18
C19
C20
C21
weekday
hour_
CPU times: user 10min 5s, sys: 3.62 s, total: 10min 9s
Wall time: 10min 9s


In [13]:
print("there are total : ",count," category in the training set")
print(count==len(hash_dict.keys()))

there are total :  9449236  category in the training set
True


In [14]:
%%time
for i in column2handle:
    print(i)
    X_test[i+'_hash']=X_test[i].apply(lambda x :get_hash_value_dict(i+'_'+str(x)))

C1
banner_pos
site_id
site_domain
site_category
app_id
app_domain
app_category
device_id
device_ip
device_model
device_type
device_conn_type
C14
C15
C16
C17
C18
C19
C20
C21
weekday
hour_
CPU times: user 1min 8s, sys: 148 ms, total: 1min 8s
Wall time: 1min 8s


In [15]:
print("there are total : ",count," category in the training& testing set")
print("there are additional : ",count-9449236," category from testing set")
print(count==len(hash_dict.keys()))

there are total :  10269200  category in the training& testing set
there are additional :  819964  category from testing set
True


In [16]:
X_train[['click']+[ i+'_hash' for i in column2handle]].to_pickle('train_df_hash')

In [17]:
X_test[[ i+'_hash' for i in column2handle]].to_pickle('test_df_hash')

In [18]:
import numpy as np

# Save
np.save('hash_dict.npy', hash_dict) 

# Load
# read_dictionary = np.load('hash_dict.npy').item()


# [Sec1] Building FF with Pytorch

In [ ]:
import pandas as pd 
X_train = pd.read_pickle('train_df_hash')
X_test = pd.read_pickle('test_df_hash')

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
#we need a 10269200 embedding matrix, let k be 100
embedding = nn.Embedding(10269200, 100)

In [ ]:
# give a length 23 array
input = Variable(torch.LongTensor(X_train.iloc[0,1:].values))
# get the corresponds matrix
W = embedding(input)


what we want is 
$$
\phi_{FM}(w,x) =  \Sigma_{j_1,j_2 \epsilon C_{2}}   <w_{j1} , w_{j2}>  * x_{j1} * x_{j2}  -(1)
$$
$w_{j1}$and $w_{j2}$ are two vectors with length k

and our objective function is :
$$
min_{w}  \Sigma_{i=1}^{n} ( log(1+exp(-yi * \phi(w,x_{i}) )) ) + \frac{\lambda}{2} * ||w||^{2} -(2) 
$$

since C2 is a large number : 10269200

for each sample i there acctually will be 23 $w_{j}$ will be involved in (1) 


now we extract the $w_{j}$ with corresponse sample i  have a matrix which is 
$$
W_{i} = \left(\begin{array}{cc} 
w_{11} & ... w_{1k}\\
w_{21} & ... w_{2k}\\
...\\
w_{f1} & ... w_{fk}\\
\end{array}\right)
$$ 

Let
$$
A = W_{i} * W_{i}'
$$

therefore
$$
\phi_{FM}(w,x_{i}) =  (sum(A) + trace(A))/2
$$


In [ ]:
W.size()

In [ ]:
A = torch.mm(W,torch.t(W))
A.sum()+ A.trace()

In [ ]:
X_train['click'][0]

# [Sec1-1] now make a nn.module for FM

In [1]:
import pandas as pd 
X_train = pd.read_pickle('train_df_hash')
X_test = pd.read_pickle('test_df_hash')

In [3]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
# create dataloader
msk = np.random.rand(len(X_train)) < 0.8
train_data = torch.LongTensor(X_train.iloc[msk,1:].values)
train_labels = torch.from_numpy(X_train.iloc[msk,0].values).float()

test_data = torch.LongTensor(X_train.iloc[~msk,1:].values)
test_labels = torch.from_numpy(X_train.iloc[~msk,0].values).float()


In [4]:
# create our dataloader first 
import torch.utils.data as data_utils

train = data_utils.TensorDataset(train_data, train_labels)
train_loader = data_utils.DataLoader(train, batch_size=1, shuffle=True,num_workers=8)

test = data_utils.TensorDataset(test_data, test_labels)
test_loader = data_utils.DataLoader(test, batch_size=1, shuffle=False,num_workers=8)

In [5]:
class FF(nn.Module):
    def __init__(self,k=50):
        super(FF, self).__init__()
        self.embedding = nn.Embedding(10269200, k, sparse=True)
        init_mat = np.random.normal(0, 0.0001, (10269200,k)) # default embedding variance too large
        self.embedding.weight = nn.Parameter(torch.from_numpy(init_mat).float())
        self.bias = nn.Parameter(torch.ones(1)*0.18606) # let initial prob is close to 0.1698(mean of train click)
        self.D = nn.Parameter(torch.ones(23,23))
        self.dropout = nn.Dropout(p=0.5)
    def forward(self, x):
        W = self.embedding(x[0])
        A = torch.mm(W,torch.t(W)) * self.D
        A = self.dropout(A)
        return A

In [6]:
class StableBCELoss(torch.nn.modules.Module):
    def __init__(self):
        super(StableBCELoss, self).__init__()
    def forward(self, input, target,alpha=0,l2_term=0):
        neg_abs = - input.abs()
        loss = input.clamp(min=0) - input * target + (1 + neg_abs.exp()).log()
        if(alpha>0):
            return loss.mean()+alpha*l2_term
        return loss.mean()

In [7]:
import torch.optim as optim
net = FF()
net.cuda()
criterion = StableBCELoss()

In [8]:
optimizer = optim.Adagrad(net.parameters(), lr=1e-2)

In [9]:
#net.load_state_dict(torch.load('FF_epoch{}.pth'.format(1)))

In [ ]:
min_testing_loss = 100
#min_testing_loss = 0
for epoch in range(10):  # loop over the dataset multiple times
    running_loss = 0.0
    net.train()
    for i, data in enumerate(train_loader, 0):
        # get the inputs
        inputs, labels = data
        # wrap them in Variable
        inputs, labels = Variable(inputs.cuda()), Variable(labels.float().cuda())

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        A = net(inputs)
        outputs = (A.sum()+ A.trace()).view(-1, 1)+net.bias
        loss = criterion(outputs, labels.view(-1, 1),1e-3,A.trace())
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 10000 == 9999:    # print every 10000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 10000))
            running_loss = 0.0
            
    #============= eval testing ================
    running_loss = 0.0
    count=0
    net.eval()
    for i, data in enumerate(test_loader, 0):
        count+=1
        inputs, labels = data
        inputs, labels = Variable(inputs.cuda()), Variable(labels.float().cuda())
        
        optimizer.zero_grad()
        A = net(inputs)
        outputs = (A.sum()+ A.trace()).view(-1, 1)+net.bias
        loss = criterion(outputs, labels.view(-1, 1))
        # print statistics
        running_loss += loss.data[0]
    testing_loss = running_loss / count
    print('[%d, %5d] testing loss: %.3f' %(epoch + 1, count, testing_loss))
    if min_testing_loss>testing_loss:
        torch.save(net.state_dict(), 'FF_epoch{}_{}.pth'.format(epoch,testing_loss))
        min_testing_loss = testing_loss
    running_loss = 0.0
print('Finished Training')

[2, 1230000] loss: 0.332
[2, 1240000] loss: 0.333
[2, 1250000] loss: 0.335
[2, 1260000] loss: 0.326
[2, 1270000] loss: 0.341
[2, 1280000] loss: 0.331
[2, 1290000] loss: 0.343
[2, 1300000] loss: 0.334
[2, 1310000] loss: 0.331
[2, 1320000] loss: 0.331
[2, 1330000] loss: 0.330
[2, 1340000] loss: 0.333
[2, 1350000] loss: 0.340
[2, 1360000] loss: 0.325
[2, 1370000] loss: 0.324
[2, 1380000] loss: 0.328
[2, 1390000] loss: 0.322
[2, 1400000] loss: 0.320
[2, 1410000] loss: 0.331
[2, 1420000] loss: 0.329
[2, 1430000] loss: 0.333
[2, 1440000] loss: 0.339
[2, 1450000] loss: 0.327
[2, 1460000] loss: 0.330
[2, 1470000] loss: 0.338
[2, 1480000] loss: 0.329
[2, 1490000] loss: 0.331
[2, 1500000] loss: 0.331
[2, 1510000] loss: 0.328
[2, 1520000] loss: 0.330
[2, 1530000] loss: 0.324
[2, 1540000] loss: 0.329
[2, 1550000] loss: 0.325
[2, 1560000] loss: 0.329
[2, 1570000] loss: 0.334
[2, 1580000] loss: 0.333
[2, 1590000] loss: 0.334
[2, 1600000] loss: 0.326
[2, 1610000] loss: 0.325
[2, 1620000] loss: 0.335


In [ ]:
[1, 8083436] testing loss: 0.375


In [ ]:
torch.save(net.state_dict(), 'FF_epoch{}_{}.pth'.format(1,min_testing_loss))

In [ ]:
#[1, 8084727] testing loss: 0.375

# torch.save(net.state_dict(), 'FF_epoch{}_{}.pth'.format(1,0.373))


# [Sec1-2] apply on testing Set

In [ ]:
def prob(x):
    a=np.exp(x)
    return a/(1+a)

In [ ]:
#net.load_state_dict(torch.load('FF_epoch{}.pth'.format(1)))

A = np.array([])
for i in range(len(X_test)):
    input = X_test.values[i]
    input = Variable(torch.LongTensor(input))
    input = input.contiguous().view(1,-1).cuda()
    out = net(input)
    prob_out = prob(out.data.cpu().numpy()[0][0])
    A = np.append(A, prob_out)

In [ ]:
import pandas as pd
sampleSubmission = pd.read_csv("data/sampleSubmission")
sampleSubmission.click = A
# with a naive feature hashing (0.408, 1142/1604)
sampleSubmission.to_csv("FF_pytorch_20170826.csv",index=False)

# [Sec2] Build a model for Batch processing

In [ ]:
# with using torch.bmm

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [ ]:
embedding = nn.Embedding(10, 3)

In [ ]:
print(torch.mm(torch.t(a),a))
print(torch.mm(torch.t(a),a).sum())
